<a href="https://colab.research.google.com/github/sumithdcosta/Python/blob/master/framework/framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir /framework
!mkdir /framework/dev
!mkdir /framework/dev/project
!mkdir /framework/dev/project/conf/
!mkdir /framework/dev/project/bin/
!mkdir /framework/dev/project/models/
!mkdir /framework/dev/project/lib/
!mkdir /framework/dev/project/lib/utils
!mkdir /framework/dev/project/lib/db
!touch /framework/dev/project/bin/sourceFileCopy.py
!chmod 775 /framework/dev/project/bin/sourceFileCopy.py
!mkdir /framework/dev/project/data/
!mkdir /framework/dev/project/data/in/
!mkdir /framework/dev/project/data/out/
!mkdir /framework/dev/project/data/processing/
!mkdir /framework/dev/project/data/reject/
!echo "Hello World" > /framework/dev/project/data/in/testsourcefile.csv

In [2]:
%%writefile /framework/dev/frameWork.ksh
if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/dev/'
fi 

if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/qa/'
fi 

Writing /framework/dev/frameWork.ksh


In [3]:
%%writefile /framework/dev/project/conf/project.ksh
if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/dev/project'
fi 

if [[ $1 = *[dD][eE][vV]* ]]; then
  export PATH = '/framework/qa/project'
fi 


Writing /framework/dev/project/conf/project.ksh


In [4]:
%%writefile /framework/dev/project/bin/genExecute.ksh
#!/bin/ksh

#check for Environment and running profiles
dirName=`dirname $0`

if [[ $dirName = *[dD][eE][vV]* ]]
  then
  export environment="DEV"
  echo $environment
elif [[ $dirName = *[qQ][aQ]* ]]
  then
  export environment="QA"
  echo $environment
else
  export environment="UNKNOWN"
  echo $environment
  exit 6
fi 

CONFPATH=`echo $dirName|sed -e 's/bin/conf/g'`
if [ ! -f "${CONFPATH}/project.ksh" ];then
  echo "${CONFPATH}/project.ksh is missing"
  exit 7
fi
echo "sourcing ${CONFPATH}/project.ksh"
. ${CONFPATH}/project.ksh $environment 2>/dev/null 1>/dev/null

CONFPATH=`echo $dirName|sed -e 's/project\/bin//g'`
if [ ! -f "${CONFPATH}/frameWork.ksh" ];then
  echo "${CONFPATH}/frameWork.ksh is missing"
  exit 7
fi
echo "sourcing ${CONFPATH}/frameWork.ksh"
. ${CONFPATH}/frameWork.ksh $environment 2>/dev/null 1>/dev/null

echo $(dirname $0)/$*
$(dirname $0)/$*


Writing /framework/dev/project/bin/genExecute.ksh


In [118]:
%%writefile /framework/dev/project/conf/sourceFileCopyConf.json
{
    "1":{
        "step-types":{
            "FILE_COPY":[{
                "templateName":"SOURCE_FILE_COPY",
                "SOURCE":{
                    "Parameters":[{
                        "SOURCE_PATH":"/framework/dev/project/data/in/",
                        "SOURCE_FILE":"testsourcefile.csv"
                    }]
                },
                "TARGET":{
                    "Parameters":[{
                        "TARGET_PATH":"/framework/dev/project/data/processing/"                        
                    }]
                }
            }]
        }
    }
}

Overwriting /framework/dev/project/conf/sourceFileCopyConf.json


In [119]:
%%writefile /framework/dev/project/conf/sourceFileCopyAppendTimeStampConf.json
{
    "1":{
        "step-types":{
            "FILE_COPY":[{
                "templateName":"SOURCE_FILE_COPY",
                "SOURCE":{
                    "Parameters":[{
                        "SOURCE_PATH":"/framework/dev/project/data/in/",
                        "SOURCE_FILE":"testsourcefile.csv"
                    }]
                },
                "TARGET":{
                    "Parameters":[{
                        "TARGET_PATH":"/framework/dev/project/data/processing/",
                        "APPEND_TIMESTAMP":"True"
                    }]
                }
            }]
        }
    }
}

Overwriting /framework/dev/project/conf/sourceFileCopyAppendTimeStampConf.json


In [120]:
%%writefile /framework/dev/project/conf/sourceFileMoveConf.json
{
    "1":{
        "step-types":{
            "FILE_MOVE":[{
                "templateName":"SOURCE_FILE_MOVE",
                "SOURCE":{
                    "Parameters":[{
                        "SOURCE_PATH":"/framework/dev/project/data/in/",
                        "SOURCE_FILE":"testsourcefile.csv"
                    }]
                },
                "TARGET":{
                    "Parameters":[{
                        "TARGET_PATH":"/framework/dev/project/data/processing/"                        
                    }]
                }
            }]
        }
    }
}

Overwriting /framework/dev/project/conf/sourceFileMoveConf.json


In [121]:
%%writefile /framework/dev/project/conf/sourceFileMoveAppendTimeStampConf.json
{
    "1":{
        "step-types":{
            "FILE_MOVE":[{
                "templateName":"SOURCE_FILE_MOVE",
                "SOURCE":{
                    "Parameters":[{
                        "SOURCE_PATH":"/framework/dev/project/data/in/",
                        "SOURCE_FILE":"testsourcefile.csv"
                    }]
                },
                "TARGET":{
                    "Parameters":[{
                        "TARGET_PATH":"/framework/dev/project/data/processing/",
                        "APPEND_TIMESTAMP":"True"
                    }]
                }
            }]
        }
    }
}

Overwriting /framework/dev/project/conf/sourceFileMoveAppendTimeStampConf.json


In [122]:
%%writefile /framework/dev/project/lib/utils/__init__.py
from lib.utils.unixOperations import unixOperations

Overwriting /framework/dev/project/lib/utils/__init__.py


In [142]:
%%writefile /framework/dev/project/lib/utils/unixOperations.py
#!/usr/local/bin/python
import logging
import glob
import shutil
import sys
import os
from datetime import datetime
import zipfile
from os.path import join

lib_path = os.path.abspath(os.path.join(__file__,'..','..'))
sys.path.append(lib_path)

class unixOperations():

  def __init__(self):
    self._log = logging.getLogger('unixOperations')

  def __del__(self):
    pass

  def _on_start(self):
    pass

  def _on_stop(self):
    pass

  def getFileList(self, rawFileName):
    self.rawFileName = rawFileName
    self.fileList = [files for files in glob.glob(self.rawFileName)]
    return self.fileList

  def touchFile(self, outdir, outfile):
    self._log.info('touch file : %s%s' %(outdir, outfile))
    open(outdir+'/'+outfile, 'a').close()
    os.utime(outdir+'/'+outfile, None)

  def fileCopy(self, indir, infile, outdir, outfile):
    self.touchFile(outdir, outfile)
    self._log.info('copy file: %s/%s to %s/%s' %(indir, infile, outdir, outfile))
    shutil.copy(indir+'/'+infile, outdir+'/'+outfile)
    self._log.info('Copy Completed')

  def copyFileToDirectory(self, absFileNameToCopy, targetPath):
    self.absFileNameToCopy = absFileNameToCopy
    self.targetPath = targetPath
    if not os.path.exists(self.targetPath):
      os.makedirs(self.targetPath)
    if os.path.isfile(self.absFileNameToCopy):
      self._log.info("Copy file from :" + self.absFileNameToCopy + " To " + self.targetPath)
      shutil.copy2((self.absFileNameToCopy),(self.targetPath))
    else:
      self._log.info("Error while Copying file :" + self.absFileNameToCopy)

  def copyFile(self, absFileNameToCopy, absTargetFileName):
    self.absFileNameToCopy = absFileNameToCopy
    self.absTargetFileName = absTargetFileName
    if os.path.isfile(self.absFileNameToCopy):
      self._log.info("Copy file from :" + self.absFileNameToCopy + " To " + self.absTargetFileName)
      shutil.copy2((self.absFileNameToCopy),(self.absTargetFileName))
    else:
      self._log.info("Error while Copying file :" + self.absFileNameToCopy)

  def moveFileToDirectory(self, absFileNameToMove, targetPath):
    self.absFileNameToMove = absFileNameToMove
    self.targetPath = targetPath
    if not os.path.exists(self.targetPath):
      os.makedirs(self.targetPath)
    if os.path.isfile(self.absFileNameToMove):
      self._log.info("Move file from :" + self.absFileNameToMove + " To " + self.targetPath)
      shutil.copy2((self.absFileNameToMove),(self.targetPath))
      os.remove(self.absFileNameToMove)
    else:
      self._log.info("Error while Moving file :" + self.absFileNameToMove)

  def moveFile(self, absFileNameToMove, absTargetFileName):
    self.absFileNameToMove = absFileNameToMove
    self.absTargetFileName = absTargetFileName
    if os.path.isfile(self.absFileNameToMove):
      self._log.info("Move file from :" + self.absFileNameToMove + " To " + self.absTargetFileName)
      shutil.copy2((self.absFileNameToMove),(self.absTargetFileName))
      os.remove(self.absFileNameToMove)
    else:
      self._log.info("Error while Moving file :" + self.absFileNameToMove)




Overwriting /framework/dev/project/lib/utils/unixOperations.py


In [143]:
%%writefile /framework/dev/project/models/__init__.py
from models.fileCopy import fileCopy
from models.fileMove import fileMove

Overwriting /framework/dev/project/models/__init__.py


In [125]:
%%writefile /framework/dev/project/models/fileCopy.py
#!/usr/local/bin/python
import logging
import glob
import shutil
import sys
import os
from os.path import isfile, join
from datetime import datetime

lib_path = os.path.abspath(os.path.join(__file__,'..','..'))
sys.path.append(lib_path)

from lib.utils.unixOperations import unixOperations

class fileCopy():

  def __init__(self):
    self._log = logging.getLogger('fileCopy')

  def __del__(self):
    pass
  
  def _on_start(self):
    pass

  def _on_stop():
    pass
  
  def copy(self, configData):
    self.configData = configData
    self._log.debug('copy configData: %s', (configData))

    #reading source information from JSON file
    self.sourceFileParameters = self.configData["SOURCE"]["Parameters"]
    self._log.info('sourceFileParameters : %s ', (self.sourceFileParameters))

    #reading target information from JSON file
    self.targetFileParameters = self.configData["TARGET"]["Parameters"]
    self._log.info('targetFileParameters : %s ', (self.targetFileParameters))

    #building source list
    self.sourceFileList = unixOperations().getFileList(join(self.sourceFileParameters[0]['SOURCE_PATH'],\
                                                            self.sourceFileParameters[0]["SOURCE_FILE"]))

    #processing file from List
    if not self.sourceFileList:
      self._log.info('sourceFileList is empty')
    elif self.targetFileParameters[0].get('APPEND_TIMESTAMP',False):
      self.currentTimeStamp = datetime.now().strftime("%Y%m%d%H%M%S")
      for sourceFile in self.sourceFileList:
        self.sourceFileNameSplit = os.path.split(sourceFile)
        self.sourceFileNamePrefix = str(os.path.splitext(self.sourceFileNameSplit[1])[0])
        self.sourceFileNamePostfix = str(os.path.splitext(self.sourceFileNameSplit[1])[1])
        self.targetFileName = self.sourceFileNamePrefix.strip() +"_"+ str(self.currentTimeStamp) + self.sourceFileNamePostfix.strip()
        unixOperations().copyFile(sourceFile, join(self.targetFileParameters[0]["TARGET_PATH"], self.targetFileName))
    else:
      for sourceFile in self.sourceFileList:
        unixOperations().copyFileToDirectory(sourceFile, self.targetFileParameters[0]["TARGET_PATH"])






Overwriting /framework/dev/project/models/fileCopy.py


In [126]:
%%writefile /framework/dev/project/models/fileMove.py
#!/usr/local/bin/python
import logging
import glob
import shutil
import sys
import os
from os.path import isfile, join
from datetime import datetime

lib_path = os.path.abspath(os.path.join(__file__,'..','..'))
sys.path.append(lib_path)

from lib.utils.unixOperations import unixOperations

class fileMove():

  def __init__(self):
    self._log = logging.getLogger('fileMove')

  def __del__(self):
    pass
  
  def _on_start(self):
    pass

  def _on_stop():
    pass
  
  def move(self, configData):
    self.configData = configData
    self._log.debug('copy configData: %s', (configData))

    #reading source information from JSON file
    self.sourceFileParameters = self.configData["SOURCE"]["Parameters"]
    self._log.info('sourceFileParameters : %s ', (self.sourceFileParameters))

    #reading target information from JSON file
    self.targetFileParameters = self.configData["TARGET"]["Parameters"]
    self._log.info('targetFileParameters : %s ', (self.targetFileParameters))

    #building source list
    self.sourceFileList = unixOperations().getFileList(join(self.sourceFileParameters[0]['SOURCE_PATH'],\
                                                            self.sourceFileParameters[0]["SOURCE_FILE"]))

    #processing file from List
    if not self.sourceFileList:
      self._log.info('sourceFileList is empty')
    elif self.targetFileParameters[0].get('APPEND_TIMESTAMP',False):
      self.currentTimeStamp = datetime.now().strftime("%Y%m%d%H%M%S")
      for sourceFile in self.sourceFileList:
        self.sourceFileNameSplit = os.path.split(sourceFile)
        self.sourceFileNamePrefix = str(os.path.splitext(self.sourceFileNameSplit[1])[0])
        self.sourceFileNamePostfix = str(os.path.splitext(self.sourceFileNameSplit[1])[1])
        self.targetFileName = self.sourceFileNamePrefix.strip() +"_"+ str(self.currentTimeStamp) + self.sourceFileNamePostfix.strip()
        unixOperations().moveFile(sourceFile, join(self.targetFileParameters[0]["TARGET_PATH"], self.targetFileName))
    else:
      for sourceFile in self.sourceFileList:
        unixOperations().moveFileToDirectory(sourceFile, self.targetFileParameters[0]["TARGET_PATH"])






Overwriting /framework/dev/project/models/fileMove.py


In [132]:
%%writefile /framework/dev/project/bin/sourceFileCopy.py
#!/usr/local/bin/python
#generic script to caopy the source data
#Name           : sourceFileCopy.py
#Descritpion    : script copies/move file from source directory
#
#/../../sourceFileCopy.ksh -cfg sourceFileCopyConfig.json
#History
#Version      | Date        | Editor      |Comment
#-------------------------------------------------------------------------
#1            | 28/07/2021  | Sumith      | initial version
#
import os
import sys
import argparse
import logging
import signal
import threading
import time
from os import path
import json

lib_path = os.path.abspath(os.path.join(__file__,'..','..'))
sys.path.append(lib_path)

from models.fileCopy import fileCopy
from models.fileMove import fileMove

class sourceFileCopyBase(object):
  """base Class for task SourceFileCopy
  """
  sleep_timeout = 1
    
  def __init__(self, app_name):
    """Constructor.
    """
    self._log = logging.getLogger('')
    self._app_name = app_name
    self._stop_event = threading.Event()

  def __del__(self):
    pass

  def _on_start(self):
    """Subclass to implement what needs to run dring start of process
    """
    pass

  def _on_stop(self):
    """Subclass to implement what needs to run dring start of process
    """
    pass      

  def _install_signal_handlers(self):
    """Install signal handler for the process.
    """
    self._log.debug('Installing Signal handler')   

    def handler(signum, _):
      """signal handler.
      """
      self._log.warning('Got Signal %s', signum)
      self._on_stop()
      sys.exit(signum)   

      for sig in (signal.SIGHUP, signal.SIGINT, signal.SIGTERM,
                  signal.SIGQUIT, signal.SIGABRT):
        signal.signal(sig, handler)

  def readConfigFile(self, configFile):
    if path.exists(configFile):
      self._log.info('reading config file')
      with open(configFile) as configFile:
        configData = json.load(configFile)
      self._log.debug('configData : %s', json.dumps(configData, indent=4))
    return configData

  def _main_loop(self, configData):
    """Main Loop
    """
    for key in range(1, 99):
      try:
        (configData[str(key)])
        self._log.info('Starting step : %s',str(key))
      except KeyError:
        self._log.info('No More steps')
        break

      if 'SOURCE_FILE_COPY' in configData[str(key)]["step-types"]:
        self._log.info('Step : SOURCE_FILE_COPY')
        self.sourceCopy = sourceCopy()
        self.sourceCopy.sourceFileCopy(configData[str(key)]["step-types"]["SOURCE_FILE_COPY"][0])
        del self.sourceCopy
      if 'FILE_COPY' in configData[str(key)]["step-types"]:
        self._log.info('Step : FILE_COPY')
        self.fileCopy = fileCopy()
        self.fileCopy.copy(configData[str(key)]["step-types"]["FILE_COPY"][0])
        del self.fileCopy
      if 'FILE_MOVE' in configData[str(key)]["step-types"]:
        self._log.info('Step : FILE_MOVE')
        self.fileMove = fileMove()
        self.fileMove.move(configData[str(key)]["step-types"]["FILE_MOVE"][0])
        del self.fileMove              
      else:
        self._log.info('Step : None')
    

  def run(self, configFile):
    """Run application
    """
    self.configFile = configFile
    
    self._log.info('HOSTNAME : %s', os.uname()[1])
    self._log.info('USERID : %s', os.environ.get('USER'))
    self._log.info('SCRIPT PATH : %s', os.path.abspath(__file__))
    self._log.info('ConfigFile : %s', self.configFile)

    exit_status = 0
    self._install_signal_handlers()

    try:
      self._on_start()
      configData = self.readConfigFile(configFile)
      self._main_loop(configData)
    except BaseException as exc:
      self._log.exception('Unrecoverable exception in %s main loop.'
                          'Exiting: %s', self._app_name, exc)
      exit_status = 1
    finally:
      self._on_stop()
      self._log.info('Done.')
    return exit_status
    

def main(config):

  class sourceFileCopy(sourceFileCopyBase):
    """initializing base call
    """

  if (config.debug):
    logging.basicConfig(format='%(asctime)s:%(filename)s[%(lineno)d]:%(process)d:%(levelname)s: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.DEBUG)
  else:
    logging.basicConfig(format='%(asctime)s:%(filename)s[%(lineno)d]:%(process)d:%(levelname)s: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)

  configFile = config.configfile
  returnCode = sourceFileCopy('SourceFileCopy').run(configFile)
  if returnCode != 0:
    sys.exit(1)


if __name__ == '__main__':
  parser = argparse.ArgumentParser(description='''Description : Source File copy ''',
                                   formatter_class=argparse.RawDescriptionHelpFormatter,
                                   epilog='''USAGE; {0} -cfg [config] '''.format(os.path.basename(sys.argv[0])))
  
  parser.add_argument('--configfile', '-c',
                      help='Config json file',
                      required=True)
  
  parser.add_argument('--debug', '-v',
                      action='store_true',
                      help='Debug/Verbose Mode')
  
  config = parser.parse_args(sys.argv[1:])

  main(config)

Overwriting /framework/dev/project/bin/sourceFileCopy.py


In [133]:
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json
08/03/2021 11:00:29 AM:sourceFileCopy.py[112]:2169:INFO: HOSTNAME : 60f9e801d1d1
08/03/2021 11:00:29 AM:sourceFileCopy.py[113]:2169:INFO: USERID : None
08/03/2021 11:00:29 AM:sourceFileCopy.py[114]:2169:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
08/03/2021 11:00:29 AM:sourceFileCopy.py[115]:2169:INFO: ConfigFile : /framework/dev/project/conf/sourceFileCopyConf.json
08/03/2021 11:00:29 AM:sourceFileCopy.py[71]:2169:INFO: reading config file
08/03/2021 11:00:29 AM:sourceFileCopy.py[83]:2169:INFO: Starting step : 1
08/03/2021 11:00:29 AM:sourceFileCopy.py[94]:2169:INFO: Step : FILE_COPY
08/03/2021 11:00:29 AM:fileCopy.py[35]:2169:INFO: sourceFileParameters : [{'SOURCE_PATH': '/framework/dev/project/data/in/', 'SOURCE_FILE': 'testsourcefile.csv'}] 
08/03/2021 11:00:29 AM:fi

In [134]:
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyAppendTimeStampConf.json

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyAppendTimeStampConf.json
08/03/2021 11:00:30 AM:sourceFileCopy.py[112]:2187:INFO: HOSTNAME : 60f9e801d1d1
08/03/2021 11:00:30 AM:sourceFileCopy.py[113]:2187:INFO: USERID : None
08/03/2021 11:00:30 AM:sourceFileCopy.py[114]:2187:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
08/03/2021 11:00:30 AM:sourceFileCopy.py[115]:2187:INFO: ConfigFile : /framework/dev/project/conf/sourceFileCopyAppendTimeStampConf.json
08/03/2021 11:00:30 AM:sourceFileCopy.py[71]:2187:INFO: reading config file
08/03/2021 11:00:30 AM:sourceFileCopy.py[83]:2187:INFO: Starting step : 1
08/03/2021 11:00:30 AM:sourceFileCopy.py[94]:2187:INFO: Step : FILE_COPY
08/03/2021 11:00:30 AM:fileCopy.py[35]:2187:INFO: sourceFileParameters : [{'SOURCE_PATH': '/framework/dev/project/data/in/', 'SOURCE_FILE': 'testsourcefile.csv

In [145]:
!echo "Hello World move file to directory" > /framework/dev/project/data/in/testsourcefile.csv
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileMoveConf.json
!ls -lrt /framework/dev/project/data/in/testsourcefile.csv

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileMoveConf.json
08/03/2021 11:08:22 AM:sourceFileCopy.py[112]:2413:INFO: HOSTNAME : 60f9e801d1d1
08/03/2021 11:08:22 AM:sourceFileCopy.py[113]:2413:INFO: USERID : None
08/03/2021 11:08:22 AM:sourceFileCopy.py[114]:2413:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
08/03/2021 11:08:22 AM:sourceFileCopy.py[115]:2413:INFO: ConfigFile : /framework/dev/project/conf/sourceFileMoveConf.json
08/03/2021 11:08:22 AM:sourceFileCopy.py[71]:2413:INFO: reading config file
08/03/2021 11:08:22 AM:sourceFileCopy.py[83]:2413:INFO: Starting step : 1
08/03/2021 11:08:22 AM:sourceFileCopy.py[99]:2413:INFO: Step : FILE_MOVE
08/03/2021 11:08:22 AM:fileMove.py[35]:2413:INFO: sourceFileParameters : [{'SOURCE_PATH': '/framework/dev/project/data/in/', 'SOURCE_FILE': 'testsourcefile.csv'}] 
08/03/2021 11:08:22 AM:fi

In [146]:
!echo "Hello World !! move file to diretcory with time stamp" > /framework/dev/project/data/in/testsourcefile.csv
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileMoveAppendTimeStampConf.json
!ls -lrt /framework/dev/project/data/in/testsourcefile.csv

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileMoveAppendTimeStampConf.json
08/03/2021 11:08:38 AM:sourceFileCopy.py[112]:2437:INFO: HOSTNAME : 60f9e801d1d1
08/03/2021 11:08:38 AM:sourceFileCopy.py[113]:2437:INFO: USERID : None
08/03/2021 11:08:38 AM:sourceFileCopy.py[114]:2437:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
08/03/2021 11:08:38 AM:sourceFileCopy.py[115]:2437:INFO: ConfigFile : /framework/dev/project/conf/sourceFileMoveAppendTimeStampConf.json
08/03/2021 11:08:38 AM:sourceFileCopy.py[71]:2437:INFO: reading config file
08/03/2021 11:08:38 AM:sourceFileCopy.py[83]:2437:INFO: Starting step : 1
08/03/2021 11:08:38 AM:sourceFileCopy.py[99]:2437:INFO: Step : FILE_MOVE
08/03/2021 11:08:38 AM:fileMove.py[35]:2437:INFO: sourceFileParameters : [{'SOURCE_PATH': '/framework/dev/project/data/in/', 'SOURCE_FILE': 'testsourcefile.csv

In [19]:
!/bin/bash /framework/dev/project/bin/genExecute.ksh sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json --debug

DEV
sourcing /framework/dev/project/conf/project.ksh
sourcing /framework/dev//frameWork.ksh
/framework/dev/project/bin/sourceFileCopy.py --config /framework/dev/project/conf/sourceFileCopyConf.json --debug
08/03/2021 09:10:23 AM:sourceFileCopy.py[109]:654:INFO: HOSTNAME : 60f9e801d1d1
08/03/2021 09:10:23 AM:sourceFileCopy.py[110]:654:INFO: USERID : None
08/03/2021 09:10:23 AM:sourceFileCopy.py[111]:654:INFO: SCRIPT PATH : /framework/dev/project/bin/sourceFileCopy.py
08/03/2021 09:10:23 AM:sourceFileCopy.py[112]:654:INFO: ConfigFile : /framework/dev/project/conf/sourceFileCopyConf.json
08/03/2021 09:10:23 AM:sourceFileCopy.py[53]:654:DEBUG: Installing Signal handler
08/03/2021 09:10:23 AM:sourceFileCopy.py[68]:654:INFO: reading config file
08/03/2021 09:10:23 AM:sourceFileCopy.py[71]:654:DEBUG: configData : {
    "1": {
        "step-types": {
            "FILE_COPY": [
                {
                    "templateName": "SOURCE_FILE_COPY",
                    "SOURCE": {
            

In [10]:
!cat /framework/dev/project/data/in/testsourcefile.csv

Hello World


In [11]:
ls -rlt /framework/dev/project/conf/sourceFileCopyConf.json

-rw-r--r-- 1 root root 617 Jul 30 10:26 /framework/dev/project/conf/sourceFileCopyConf.json
